In [ ]:
# Kaggle setup
import sys
sys.path.append('/kaggle/working/python_packages')

# Install packages 
!pip install numpy==1.26.4 opencv-python ultralytics -t /kaggle/working/python_packages


In [7]:
import sys
sys.path.append('/kaggle/working/python_packages')

In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

print("NumPy:", np.__version__)
print("OpenCV:", cv2.__version__)
print("✅ Ultralytics: YOLO imported successfully!")  # ultralytics.__version__ doesn't exist



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
NumPy: 2.0.2
OpenCV: 4.12.0
✅ Ultralytics: YOLO imported successfully!


In [13]:
!rm -rf /kaggle/working/lstm_dataset
!rm -rf /kaggle/working/lstm_dataset.zip
!rm -rf  /kaggle/working/runs
!rm -rf /kaggle/working/output
!rm -rf /kaggle/working/skeleton-images.zip
!rm -rf /kaggle/working/yolo11s-pose.pt
!ls -la /kaggle/working/



total 16
drwxr-xr-x   4 root root 4096 Jan 31 17:08 .
drwxr-xr-x   5 root root 4096 Jan 31 15:50 ..
drwxr-xr-x 112 root root 4096 Jan 31 15:50 python_packages
drwxr-xr-x   2 root root 4096 Jan 31 15:50 .virtual_documents


# Skeleton Extraction For LSTM Input


In [1]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>LSTM DATA EXTRACTION<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ===================== IMPORTS =====================
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import defaultdict

# ===================== DEVICE =====================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ===================== OUTPUT ROOT =====================
OUTPUT_ROOT = "/kaggle/working/lstm_dataset"

# ===================== VIDEO PATHS =====================
VIDEO_PATHS = {
    "normal": [
        "/kaggle/input/exam-videos/Normal_1st.mp4",
        "/kaggle/input/exam-videos/Normal_2nd.mp4",
        "/kaggle/input/exam-videos/Normal_3rd.mp4",
        "/kaggle/input/exam-videos/Normal_4th.mp4",
    ],
    "suspicious": [
        "/kaggle/input/exam-videos/Sus_1st.mp4",
        "/kaggle/input/exam-videos/Sus_2nd.mp4",
        "/kaggle/input/exam-videos/Sus_3rd.mp4",
        "/kaggle/input/exam-videos/Sus_4th.mp4",
    ]
}

# ===================== PARAMETERS =====================
IMG_SIZE = 384
ROI_SIZE = 224
FPS_SAMPLE = 2
KPT_CONF_TH = 0.5

BODY_BONE_THICKNESS = 3
HEAD_BONE_THICKNESS = 2

MAX_STUDENTS = 3  # ← Expected number of students per video

# ===================== MODEL =====================
model = YOLO("yolo11s-pose.pt").to(device)

# ===================== COCO UPPER BODY =====================
UPPER_BODY_KPTS = {0,1,2,3,4,5,6,7,8,9,10,11,12}

SKELETON = [
    (0,1),(0,2),(1,3),(2,4),
    (0,5),(0,6),(5,6),
    (5,7),(7,9),
    (6,8),(8,10),
    (5,11),(6,12),(11,12)
]

# ===================== VALID SKELETON CHECK =====================
def is_valid_skeleton(conf):
    return (
        conf[0] > KPT_CONF_TH and
        conf[5] > KPT_CONF_TH and
        conf[6] > KPT_CONF_TH and
        (conf[11] > KPT_CONF_TH or conf[12] > KPT_CONF_TH)
    )

# ===================== ID STABILIZER =====================
class IDStabilizer:
    def __init__(self, max_students=3):
        self.max_students = max_students
        self.id_map = {}  # original_id -> stable_id
        self.next_stable_id = 0
        self.id_frames = defaultdict(int)  # Count frames per ID
    
    def get_stable_id(self, original_id):
        """Map unstable tracker ID to stable student ID"""
        if original_id not in self.id_map:
            if self.next_stable_id < self.max_students:
                self.id_map[original_id] = self.next_stable_id
                self.next_stable_id += 1
            else:
                # If we have more IDs than expected, map to most similar existing ID
                # For now, just reject extra IDs
                return None
        
        self.id_frames[original_id] += 1
        return self.id_map[original_id]
    
    def get_stats(self):
        """Return statistics about ID usage"""
        return {
            'total_original_ids': len(self.id_map),
            'stable_ids_used': len(set(self.id_map.values())),
            'frames_per_id': dict(self.id_frames)
        }

# ===================== PROCESS =====================
for label, videos in VIDEO_PATHS.items():
    for v_idx, video_path in enumerate(videos):

        if not os.path.exists(video_path):
            print(f"❌ Missing video: {video_path}")
            continue

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"❌ Failed to open {video_path}")
            continue

        # Initialize ID stabilizer for this video
        id_stabilizer = IDStabilizer(max_students=MAX_STUDENTS)

        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_gap = max(1, fps // FPS_SAMPLE)
        frame_idx = 0
        saved = 0

        save_root = os.path.join(
            OUTPUT_ROOT,
            label,
            f"video_{v_idx+1:02d}"
        )
        os.makedirs(save_root, exist_ok=True)

        print(f"\n🎥 Processing {label}/video_{v_idx+1:02d} @ {FPS_SAMPLE} FPS")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_idx += 1
            if frame_idx % frame_gap != 0:
                continue

            results = model.track(
                frame,
                imgsz=IMG_SIZE,
                conf=0.4,
                persist=True,
                device=device,
                verbose=False,
                tracker="botsort.yaml"  # ← Better tracker for crowded scenes
            )[0]

            if results.keypoints is None or results.boxes.id is None:
                continue

            kpts_xy = results.keypoints.xy.cpu().numpy()
            kpts_conf = results.keypoints.conf.cpu().numpy()
            track_ids = results.boxes.id.cpu().numpy().astype(int)

            for tid, xy, conf in zip(track_ids, kpts_xy, kpts_conf):

                if not is_valid_skeleton(conf):
                    continue

                # Get stable ID
                stable_id = id_stabilizer.get_stable_id(tid)
                if stable_id is None:
                    continue  # Skip extra IDs beyond MAX_STUDENTS

                valid_pts = [xy[i] for i in UPPER_BODY_KPTS if conf[i] > KPT_CONF_TH]
                if len(valid_pts) < 4:
                    continue

                pts = np.array(valid_pts)
                min_x, min_y = pts.min(axis=0)
                max_x, max_y = pts.max(axis=0)

                scale = 0.9 * ROI_SIZE / max(max_x - min_x, max_y - min_y)
                cx = (min_x + max_x) / 2
                cy = (min_y + max_y) / 2

                skeleton_img = np.zeros((ROI_SIZE, ROI_SIZE), dtype=np.uint8)

                def norm(pt):
                    x = (pt[0] - cx) * scale + ROI_SIZE / 2
                    y = (pt[1] - cy) * scale + ROI_SIZE / 2
                    return int(x), int(y)

                for i, j in SKELETON:
                    if conf[i] < KPT_CONF_TH or conf[j] < KPT_CONF_TH:
                        continue

                    p1, p2 = norm(xy[i]), norm(xy[j])
                    if not (0 <= p1[0] < ROI_SIZE and 0 <= p1[1] < ROI_SIZE and
                            0 <= p2[0] < ROI_SIZE and 0 <= p2[1] < ROI_SIZE):
                        continue

                    thickness = HEAD_BONE_THICKNESS if (i == 0 or j == 0) else BODY_BONE_THICKNESS
                    cv2.line(skeleton_img, p1, p2, 255, thickness)

                skeleton_img = cv2.dilate(
                    skeleton_img, np.ones((3,3), np.uint8), iterations=1
                )

                student_dir = os.path.join(save_root, f"student_{stable_id}")
                os.makedirs(student_dir, exist_ok=True)

                cv2.imwrite(
                    os.path.join(student_dir, f"f{frame_idx:06d}.png"),
                    skeleton_img
                )
                saved += 1

        cap.release()
        
        # Print ID statistics
        stats = id_stabilizer.get_stats()
        print(f"✅ Saved {saved} skeleton frames")
        print(f"📊 ID Stats: {stats['total_original_ids']} original IDs → {stats['stable_ids_used']} stable IDs")

print("\n" + "="*60)
print("✅ SKELETON DATASET COLLECTION COMPLETE")
print(f"📁 Output: {OUTPUT_ROOT}")
print("="*60)

KeyboardInterrupt: 

# Make lstm_dataset.zip visible in kaggle/working to download the dataset

In [15]:
import shutil
import os

SRC_DIR = "/kaggle/working/lstm_dataset"
ZIP_PATH = "/kaggle/working/lstm_dataset"  # This will create lstm_dataset.zip

# Check if source directory exists
assert os.path.exists(SRC_DIR), f"Directory not found: {SRC_DIR}"

# Create zip archive
shutil.make_archive(ZIP_PATH, 'zip', SRC_DIR)

print("✅ Zip complete! Download 'lstm_dataset.zip' from Output tab")
print(f"📦 Zip file location: {ZIP_PATH}.zip")

✅ Zip complete! Download 'lstm_dataset.zip' from Output tab
📦 Zip file location: /kaggle/working/lstm_dataset.zip


# Skeleton Extraction For ResNet50v2 Input 

In [ ]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO

# ================== KAGGLE-SPECIFIC MODIFICATIONS ==================
# Set Kaggle working directory
os.chdir('/kaggle/working')

# ================== PARAMETERS ==================
IMG_SIZE = 384          # YOLO pose input
ROI_SIZE = 224          # ResNet input
FRAME_SKIP = 5
KPT_CONF_TH = 0.5

JOINT_RADIUS = 3        # (kept but NOT USED – explicitly)
BODY_BONE_THICKNESS = 3
HEAD_BONE_THICKNESS = 2

# Updated path for Kaggle input structure
VIDEO_PATH = "/kaggle/input/exam-videos/Normal_1st.mp4"

# Use Kaggle's working directory for output
OUTPUT_DIR = "/kaggle/working/skeletons"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ================== DEVICE ==================
# Kaggle has GPU available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ================== MODEL ==================
# Load model directly - Kaggle has internet access for downloads
model = YOLO("yolov8s-pose.pt").to(device)

# ================== COCO UPPER-BODY KEYPOINTS ==================
UPPER_BODY_KPTS = {0,1,2,3,4,5,6,7,8,9,10,11,12}

# ================== UPPER-BODY SKELETON ==================
SKELETON = [
    (0,1),(0,2),(1,3),(2,4),      # head
    (0,5),(0,6),(5,6),            # neck / shoulders
    (5,7),(7,9),                  # left arm
    (6,8),(8,10),                 # right arm
    (5,11),(6,12),(11,12)         # torso
]

# ================== SKELETON VALIDATION ==================
def is_valid_skeleton(conf, th):
    return (
        conf[0] > th and
        conf[5] > th and conf[6] > th and
        (conf[11] > th or conf[12] > th)
    )

# ================== VIDEO ==================
cap = cv2.VideoCapture(VIDEO_PATH)
frame_idx = 0
saved_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    if frame_idx % FRAME_SKIP != 0:
        continue

    results = model.track(
        frame,
        imgsz=IMG_SIZE,
        conf=0.4,
        persist=True,
        device=device,
        verbose=False
    )[0]

    if results.keypoints is None:
        continue

    kpts_xy = results.keypoints.xy.cpu().numpy()
    kpts_conf = results.keypoints.conf.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy()

    for pid, (xy, conf, box) in enumerate(zip(kpts_xy, kpts_conf, boxes)):

        if not is_valid_skeleton(conf, KPT_CONF_TH):
            continue

        # ================== USE ONLY UPPER BODY POINTS ==================
        valid_pts = [
            xy[i] for i in UPPER_BODY_KPTS if conf[i] > KPT_CONF_TH
        ]

        if len(valid_pts) < 4:
            continue

        pts = np.array(valid_pts)
        min_x, min_y = pts.min(axis=0)
        max_x, max_y = pts.max(axis=0)

        skel_w = max_x - min_x
        skel_h = max_y - min_y
        scale = 0.9 * ROI_SIZE / max(skel_w, skel_h)

        cx = (min_x + max_x) / 2
        cy = (min_y + max_y) / 2

        skeleton_img = np.zeros((ROI_SIZE, ROI_SIZE), dtype=np.uint8)

        def norm(pt):
            x = (pt[0] - cx) * scale + ROI_SIZE / 2
            y = (pt[1] - cy) * scale + ROI_SIZE / 2
            return int(x), int(y)

        # ================== DRAW BONES ONLY ==================
        for i, j in SKELETON:
            if conf[i] < KPT_CONF_TH or conf[j] < KPT_CONF_TH:
                continue

            if i not in UPPER_BODY_KPTS or j not in UPPER_BODY_KPTS:
                continue

            p1 = norm(xy[i])
            p2 = norm(xy[j])

            if not (0 <= p1[0] < ROI_SIZE and 0 <= p1[1] < ROI_SIZE and
                    0 <= p2[0] < ROI_SIZE and 0 <= p2[1] < ROI_SIZE):
                continue

            thickness = HEAD_BONE_THICKNESS if (i == 0 or j == 0) else BODY_BONE_THICKNESS
            cv2.line(skeleton_img, p1, p2, 255, thickness)

        # ================== POSTPROCESS ==================
        kernel = np.ones((3,3), np.uint8)
        skeleton_img = cv2.dilate(skeleton_img, kernel, iterations=1)

        filename = f"f{frame_idx:05d}_p{pid:02d}.png"
        cv2.imwrite(os.path.join(OUTPUT_DIR, filename), skeleton_img)
        saved_count += 1

    print(f"Processed frame {frame_idx}")

cap.release()
print(f"✅ DONE. Saved {saved_count} CLEAN upper-body skeleton images.")
print(f"Output saved to: {OUTPUT_DIR}")

# ZIP the dataset folder to download

In [ ]:
# At the end of your code, compress the output directory
import shutil
shutil.make_archive('/kaggle/working/skeletons_output', 'zip', '/kaggle/working/skeletons')